# Dropout
Dropout je tehnika za regularizaciju neuralnih mreža tako što se slučajno postave neke karakteristike na nulu tokom prolaza unaprijed. U ovoj vježbi treba implementirati dropout sloj i modifikovati FC mrežu da opciono koristi dropout.


In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from funkcije.fc_net import *
from funkcije.data_utils import get_CIFAR10_data
from funkcije.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from funkcije.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ Vraća relativnu grešku """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
data = get_CIFAR10_data()
for k, v in data.items():
    print('%s: ' % k, v.shape)

# Dropout prolaz unaprijed
U fajlu `funkcije/layers.py`, implementirati prolaz unaprijed za dropout. S obzirom da se dropout različito ponaša tokom treniranja i testiranja, treba implementirati operaciju za oba režima.

U ćeliji ispod testirati funkciju.

In [ ]:
np.random.seed(231)
x = np.random.randn(500, 500) + 10

for p in [0.25, 0.4, 0.7]:
    out, _ = dropout_forward(x, {'mode': 'train', 'p': p})
    out_test, _ = dropout_forward(x, {'mode': 'test', 'p': p})

    print('Test sa p = ', p)
    print('Srednja vrijednost ulaza: ', x.mean())
    print('Srednja vrijednost izlaza treniranja: ', out.mean())
    print('Srednja vrijednost izlaza testiranja: ', out_test.mean())
    print('Odnos izlaza treniranja i nule: ', (out == 0).mean())
    print('Odnos izlaza testiranja i nule: ', (out_test == 0).mean())
    print()

# Dropout prolaz unazad
U fajlu `funkcije/layers.py`, implementirati prolaz unazad za dropout. Pokretanjem ćelije ispod testirati implementaciju funkcije.

In [ ]:
np.random.seed(231)
x = np.random.randn(10, 10) + 10
dout = np.random.randn(*x.shape)

dropout_param = {'mode': 'train', 'p': 0.2, 'seed': 123}
out, cache = dropout_forward(x, dropout_param)
dx = dropout_backward(dout, cache)
dx_num = eval_numerical_gradient_array(lambda xx: dropout_forward(xx, dropout_param)[0], x, dout)

# Greška bi trebalo da bude oko e-10 ili manja
print('dx relative error: ', rel_error(dx, dx_num))

# FC mreža sa dropout-om
U fajlu `funkcije/classifiers/fc_net.py`, modifikovati implementaciju tako da se koristi dropout. Kada konstruktor mreže za `dropout` parametar primi vrijednost koja nije 1, onda mreža koristi dropout nakon scake ReLU aktivacione funkcije. U ćeliji ispod testirati implementiranu funkciju.

In [ ]:
np.random.seed(231)
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

for dropout in [1, 0.75, 0.5]:
    print('Provjera za dropout = ', dropout)
    model = FullyConnectedNet([H1, H2], input_dim=D, num_classes=C,
                            weight_scale=5e-2, dtype=np.float64,
                            dropout=dropout, seed=123)

    loss, grads = model.loss(X, y)
    print('Početni loss: ', loss)

    # Relative errors should be around e-6 or less; Note that it's fine
    # if for dropout=1 you have W2 error be on the order of e-5.
    for name in sorted(grads):
        f = lambda _: model.loss(X, y)[0]
        grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-5)
        print('%s relativna greška: %.2e' % (name, rel_error(grad_num, grads[name])))
    print()

# Regularizacioni eksperiment
Trenirati dvije dvoslojne mreže na 500 trening uzoraka: jedna neće koristiti dropout, a druga će koristi vjerovatnoću 0.25. Nakon toga vizuelizovati trening i validacione tačnosti dvije mreže tokom vremena.

In [ ]:
# Treniranje dvije identične mreže, jedna sa dropout-om, druga bez
np.random.seed(231)
num_train = 500
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

solvers = {}
dropout_choices = [1, 0.25]
for dropout in dropout_choices:
    model = FullyConnectedNet([500], dropout=dropout)
    print(dropout)

    solver = Solver(model, small_data,
                  num_epochs=25, batch_size=100,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 5e-4,
                  },
                  verbose=True, print_every=100)
    solver.train()
    solvers[dropout] = solver

In [ ]:
# Plot train and validation accuracies of the two models

train_accs = []
val_accs = []
for dropout in dropout_choices:
    solver = solvers[dropout]
    train_accs.append(solver.train_acc_history[-1])
    val_accs.append(solver.val_acc_history[-1])

plt.subplot(3, 1, 1)
for dropout in dropout_choices:
    plt.plot(solvers[dropout].train_acc_history, 'o', label='%.2f dropout' % dropout)
plt.title('Tačnost na trening podacima')
plt.xlabel('Epoha')
plt.ylabel('Tačnost')
plt.legend(ncol=2, loc='lower right')
  
plt.subplot(3, 1, 2)
for dropout in dropout_choices:
    plt.plot(solvers[dropout].val_acc_history, 'o', label='%.2f dropout' % dropout)
plt.title('Tačnost na validacionim podacima')
plt.xlabel('Epoha')
plt.ylabel('Tačnost')
plt.legend(ncol=2, loc='lower right')

plt.gcf().set_size_inches(15, 15)
plt.show()